<a href="https://colab.research.google.com/github/mdzikrim/DeepLearning/blob/main/CNN_Model_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN PyTorch

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [25]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 256), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.model(x)

model = CNN().to(device)

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(20):
    model.train()
    total_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

Epoch 1 | Loss: 1154.0193
Epoch 2 | Loss: 899.0771
Epoch 3 | Loss: 778.0177
Epoch 4 | Loss: 705.8400
Epoch 5 | Loss: 655.1831
Epoch 6 | Loss: 609.4172
Epoch 7 | Loss: 574.2173
Epoch 8 | Loss: 545.2778
Epoch 9 | Loss: 512.6494
Epoch 10 | Loss: 485.8300
Epoch 11 | Loss: 461.9132
Epoch 12 | Loss: 439.1278
Epoch 13 | Loss: 418.3021
Epoch 14 | Loss: 400.7752
Epoch 15 | Loss: 380.1481
Epoch 16 | Loss: 361.6624
Epoch 17 | Loss: 349.0241
Epoch 18 | Loss: 327.5960
Epoch 19 | Loss: 316.6558
Epoch 20 | Loss: 301.5407


In [27]:
model.eval()
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()
        preds = np.argmax(probs, axis=1)

        all_preds.extend(preds)
        all_probs.extend(probs)
        all_labels.extend(labels.cpu().numpy())

y_true = np.eye(10)[all_labels]
y_score = np.array(all_probs)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')
auc = roc_auc_score(y_true, y_score, average='macro', multi_class='ovr')

print("📊 Hasil Evaluasi:")
print(f"Akurasi  : {accuracy:.4f}")
print(f"Presisi  : {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-Score : {f1:.4f}")
print(f"AUC      : {auc:.4f}")

📊 Hasil Evaluasi:
Akurasi  : 0.7703
Presisi  : 0.7717
Recall   : 0.7703
F1-Score : 0.7706
AUC      : 0.9711


#CNN Tensorflow

In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np


In [29]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

y_train = y_train.flatten()
y_test = y_test.flatten()

y_train_onehot = tf.keras.utils.to_categorical(y_train, 10)
y_test_onehot  = tf.keras.utils.to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [35]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)),
    layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train_onehot, epochs=40, batch_size=64,
                    validation_data=(x_test, y_test_onehot))

Epoch 1/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - accuracy: 0.3239 - loss: 1.8104 - val_accuracy: 0.5810 - val_loss: 1.1698
Epoch 2/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.6036 - loss: 1.1112 - val_accuracy: 0.6720 - val_loss: 0.9270
Epoch 3/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.6985 - loss: 0.8585 - val_accuracy: 0.7211 - val_loss: 0.7953
Epoch 4/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7531 - loss: 0.7058 - val_accuracy: 0.7196 - val_loss: 0.8126
Epoch 5/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7898 - loss: 0.5987 - val_accuracy: 0.7680 - val_loss: 0.6830
Epoch 6/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8287 - loss: 0.4924 - val_accuracy: 0.7724 - val_loss: 0.6870
Epoch 7/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8512 - loss: 0.4221 - val_accuracy: 0.7828 - val_loss: 0.7099
Epoch 8/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8719 - loss: 0.3588 - val_accura

In [37]:
y_probs = model.predict(x_test)
y_preds = np.argmax(y_probs, axis=1)

accuracy  = accuracy_score(y_test, y_preds)
precision = precision_score(y_test, y_preds, average='macro')
recall    = recall_score(y_test, y_preds, average='macro')
f1        = f1_score(y_test, y_preds, average='macro')
auc       = roc_auc_score(y_test_onehot, y_probs, average='macro', multi_class='ovr')

print("📊 Hasil Evaluasi:")
print(f"Akurasi  : {accuracy:.4f}")
print(f"Presisi  : {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-Score : {f1:.4f}")
print(f"AUC      : {auc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step
📊 Hasil Evaluasi:
Akurasi  : 0.7764
Presisi  : 0.7734
Recall   : 0.7764
F1-Score : 0.7724
AUC      : 0.9697
